In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


단어 품사 구분하기  
'I work at google', 'I google at work'의 단어 품사를 구분하는 RNN 코드를 텐서플로우로 구현한다.  
각 단어는 one hot encoding으로 표현한다.

In [4]:
# I      => [1, 0, 0, 0] # 대명사
# work   => [0, 1, 0, 0] # 동사
# at     => [0, 0, 1, 0] # 전치사
# google => [0, 0, 0, 1] # 명사

# I work at google => [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
# I google at work => [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]

inputs = np.array([
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
    [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]
])
inputs

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]],

       [[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0]]])

In [11]:
tf.reset_default_graph()
tf.set_random_seed(35)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)

print('출력값 개수: {}'.format(outputs))
print('상태값 개수: {}'.format(state))
print('=' * 80)

print('가중치 개수와 편향값 개수')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
print('=' * 80)

variable_names = [v.name for v in tf.trainable_variables()]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print('출력값\n{}\n'.format(output_run))
    print('상태값\n{}\n'.format(state_run))
    print('가중치와 편향값')
    values = sess.run(variable_names)
    for k, v in zip(variable_names, values):
        print('{}\n{}'.format(k, v))

출력값 개수: Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
상태값 개수: Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
가중치 개수와 편향값 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
출력값
[[[ 0.37247857  0.27281666 -0.03415733]
  [-0.12360744 -0.23688789 -0.07885218]
  [-0.27765325 -0.14212623 -0.44862753]
  [-0.7634894  -0.13153659 -0.1189682 ]]

 [[ 0.37247857  0.27281666 -0.03415733]
  [-0.62467164  0.70083857  0.17365289]
  [ 0.10053852  0.2770306   0.05076913]
  [-0.05543748 -0.37285638 -0.04752919]]]

상태값
[[-0.7634894  -0.13153659 -0.1189682 ]
 [-0.05543748 -0.37285638 -0.04752919]]

가중치와 편향값
rnn/basic_rnn_cell/kernel:0
[[ 0.39129794  0.27990448 -0.03417063]
 [-0.13734835 -0.6881115  -0.19269955]
 [-0.19838947  0.15787274 -0.3503854 ]
 [-0.74573535  0.42230952  0.06174737]
 [-0.07638937  0.737661   -0.0250259 ]
 [ 0.22168279  0.6998607   0.48419607]
 [ 0.55390215  0.5579

첫 단어 이전에는 이전 상태값이 존재하지 않기 때문에 두 문장의 첫 단어 출력값이 동일하다.  
[ 0.37247857  0.27281666 -0.03415733]  

이전 상태값이 현재 출력값에 영향을 주기 때문에 두 번째 단어부터의 출력값은 두 문장이 다르게 나타난다.  
[-0.12360744 -0.23688789 -0.07885218]  
[-0.27765325 -0.14212623 -0.44862753]  
[-0.7634894  -0.13153659 -0.1189682 ] # 마지막 단어 출력값  

[-0.62467164  0.70083857  0.17365289]  
[ 0.10053852  0.2770306   0.05076913]  
[-0.05543748 -0.37285638 -0.04752919] # 마지막 단어 출력값  

상태값은 항상 마지막 단어의 출력값과 동일하다.  
[-0.7634894  -0.13153659 -0.1189682 ]  
[-0.05543748 -0.37285638 -0.04752919]